In [ ]:
from PIL import Image
import numpy as np
from deepclo.algorithms.algorithm_por import AlgorithmPOR, \
    rank_image_blocks as rib, sort_image_blocks
from deepclo.core.measures.measure_functions import map_measure_function, \
    Measure, Ordering
from deepclo.pipe.dataset import Dataset

dataset = Dataset(dataset_name='cifar10')
dataset.unravel()
    # sample = dataset.x_train[0]
label = dataset.y_train[0]
sample = Image.open('cu.jpg')
sample = sample.resize((1024, 1024))
sample = np.asarray(sample)
block_shape = (256, 256)
patchifier = AlgorithmPOR(sample=sample, label=label, block_shape=block_shape)
